In [15]:
# import external packages
import numpy as np
import cvxpy as cp
from sklearn.model_selection import train_test_split
import time
import math

# import internal packages
import phi_divergence as phi
from iter_gen_and_eval_alg import iter_gen_and_eval_alg
import util

The problem we examine is as follows:

Consider a company that sells $n$ different products, which it is able to produce with the use of $m$ different machines. 

The goal is to determine an optimal production plan, which specifies the amount of time $y_{jk}$ that each machine $j=1,\dots,m$ will be used for producing product $k = 1,\dotsc,n$. An optimal plan is one that maximizes the total profit of the company subject to availability constraints.

Each machine $j$ may only be used for a limited amount of time $a_j$ and incurs operating costs $c_{jk}$ per unit of product $k$ that is produced. Each unit of product $k$ can be sold at a price of $u_k$ and leftover units incur inventory holding costs of $\tilde{c}_{k}$. For this problem there are two uncertain parameters, the demand $d_k$ for each product $k$ and the quantity $p_{jk}$ of product $k$ that is produced per time unit by machine $j$. 

Putting all this together, we arrive at the following mathematical formulation:
\begin{align}
    \label{mathform:weighted_dist_1:obj}
    \min_{\mathbf{y}}&~\sum_{j=1}^{m} \sum_{k=1}^{n} c_{jk} y_{jk} + \sum_{k=1}^{n} \tilde{c}_{k} \left[ \sum_{j=1}^{m} p_{jk} y_{jk} - d_k \right]_{+} - \sum_{k=1}^{n} u_k \min \left( \sum_{j=1}^{m} p_{jk} y_{jk}, d_k \right) \\
    \text{s.t.}&~\sum_{k=1}^{n} y_{jk} \leq a_j, && j=1.\dotsc,m \label{mathform:weighted_dist_1:limit_time}\\
    &~y_{jk} \geq 0, && j=1.\dotsc,m, k=1.\dotsc,n, \label{mathform:weighted_dist_1:xVarNonNeg}
\end{align}


where $[ a ]_{+}$ is equivalent to $\max\{a,0\}$. The $\max$ and $\min$ functions make this a nonlinear optimization problem. As such, it is quite a tricky problem to deal with using conventional Robust Optimization techniques. 

Note that this problem can also be easily converted into the notation used earlier for by moving the uncertainty into the constraints and introducing supplementary variable $\theta$.
Let $\mathbf{x} = (\mathbf{y}, \theta)$ and $\mathscr{X} = \{ \mathbf{x} : \sum_{k=1}^{n} y_{jk} \leq a_j, j=1.\dotsc,m, \mathbf{y} \geq 0 \}$. We can rewrite the problem as:
\begin{align}
    \label{mathform:weighted_dist_2:obj}
    \min_{\mathbf{x} \in \mathscr{X}}&~\sum_{j=1}^{m} \sum_{k=1}^{n} c_{jk} y_{jk} + \theta  \\
    \text{s.t.}&~\sum_{k=1}^{n} \tilde{c}_{k} \left[ \sum_{j=1}^{m} p_{jk} y_{jk} - d_k \right]_{+} - \sum_{k=1}^{n} u_k \min \left( \sum_{j=1}^{m} p_{jk} y_{jk}, d_k \right) - \theta \leq 0.
\end{align}

By defining $g(\mathbf{x}) = \sum_{j=1}^{m} \sum_{k=1}^{n} c_{jk} y_{jk} + \theta$ and $f(\mathbf{x}, \mathbf{z}) = \sum_{k=1}^{n} \tilde{c}_{k} \left[ \sum_{j=1}^{m} p_{jk} y_{jk} - d_k \right]_{+} - \sum_{k=1}^{n} u_k \min \left( \sum_{j=1}^{m} p_{jk} y_{jk}, d_k \right) - \theta$ we obtain the familiar notation used throughout this paper. 

In [16]:
# problem specific functions:
def generate_unc_param_data(random_seed, N, **kwargs):
    np.random.seed(random_seed)
    scale_dim_problem = kwargs.get('scale_dim_problem', 1)
    m = 5*scale_dim_problem
    n = 10*scale_dim_problem
    
    # generate demand vector param
    d_care = np.array([25, 38, 18, 39, 60, 35, 41, 22, 74, 30])
    d_nom = ()
    for i in range(scale_dim_problem):
        d_i = d_care
        d_nom = d_nom + tuple(d_i.reshape(1, -1)[0])
    d = np.random.default_rng(seed=random_seed).dirichlet(d_nom, N) * sum(d_nom)
    
    # generate production efficiency param
    p_care = np.array([[5.0, 7.6, 3.6, 7.8, 12.0, 7.0, 8.2, 4.4, 14.8, 6.0],
                      [3.8, 5.8, 2.8, 6.0, 9.2, 5.4, 6.3, 3.4, 11.4, 4.6],
                      [2.3, 3.5, 1.6, 3.5, 5.5, 3.2, 3.7, 2.0, 6.7, 2.7],
                      [2.6, 4.0, 1.9, 4.1, 6.3, 3.7, 4.3, 2.3, 7.8, 3.2],
                      [2.4, 3.6, 1.7, 3.7, 5.7, 3.3, 3.9, 2.1, 7.0, 2.9]])
    if scale_dim_problem > 1:
        p_nom = np.block([[p_care for i in range(scale_dim_problem)] for j in range(scale_dim_problem)])
    else:
        p_nom = p_care
    p = np.random.random_sample(size = (N,m,n)) * (p_nom*1.05 - p_nom*0.95) + (p_nom*0.95)
    data = list(zip(d,p))
    data = np.array(data, dtype=object)
    return data

def get_fixed_param_data(random_seed, **kwargs):
    np.random.seed(random_seed)
    scale_dim_problem = kwargs.get('scale_dim_problem', 1)
    
    # fixed parameter values from Care (2014)
    C_care = np.array([[1.8, 2.2, 1.5, 2.2, 2.6, 2.1, 2.2, 1.7, 2.8, 1.9],
                        [1.6, 1.9, 1.3, 1.9, 2.3, 1.9, 2.0, 1.5, 2.5, 1.7],
                        [1.2, 1.5, 1.0, 1.5, 1.9, 1.4, 1.6, 1.1, 2.0, 1.3],
                        [1.3, 1.6, 1.1, 1.6, 2.0, 1.5, 1.7, 1.2, 2.2, 1.4],
                        [1.2, 1.5, 1.0, 1.6, 1.9, 1.5, 1.6, 1.1, 2.1, 1.3]])

    A_care = np.array([10, 13, 22, 19, 21])
    C_tilde_care = np.array([1.3, 1.3, 1.3, 1.3, 1.3, 1.3, 1.3, 1.3, 1.3, 1.3])
    U_care = np.array([1.5, 1.8, 1.2, 1.9, 2.2, 1.8, 1.9, 1.4, 2.4, 1.6])
    
    if scale_dim_problem > 1:
        max_deviation = 0.10 # represents max deviation from care values
        C = np.block([[np.random.random_sample(size = (5,10)) * (C_care*(1+max_deviation) - C_care*(1-max_deviation)) + (C_care*(1-max_deviation)) for i in range(scale_dim_problem)] 
                      for j in range(scale_dim_problem)])
        
        A = np.round(np.block([np.random.random_sample(size = 5) * (A_care*(1+max_deviation) - A_care*(1-max_deviation)) + (A_care*(1-max_deviation)) for i in range(scale_dim_problem)]))
        
        C_tilde = np.block([np.random.random_sample(size = 10) * (C_tilde_care*(1+max_deviation) - C_tilde_care*(1-max_deviation)) + (C_tilde_care*(1-max_deviation)) for i in range(scale_dim_problem)])
        
        U = np.block([np.random.random_sample(size = 10) * (U_care*(1+max_deviation) - U_care*(1-max_deviation)) + (U_care*(1-max_deviation)) for i in range(scale_dim_problem)])
        param_dict = {'C':C, 'A':A, 'C_tilde': C_tilde, 'U': U}
    else:
        param_dict = {'C':C_care, 'A':A_care, 'C_tilde': C_tilde_care, 'U': U_care}
    
    return param_dict

def solve_P_SCP(S, **kwargs):
    # get fixed parameter values
    C = kwargs['C']
    A = kwargs['A']
    C_tilde = kwargs['C_tilde']
    U = kwargs['U']
    
    # unzip uncertain parameters
    d,p = S.T
    
    # get dimensions of problem
    m,n = p[0].shape
    num_scen = len(d)
    
    # create variables
    theta = cp.Variable(1)
    y = cp.Variable((m, n), nonneg = True)
    
    # set up problem
    setup_time_start = time.time()
    constraints = []
    for s in range(num_scen):
        prod_s = cp.sum(cp.multiply(p[s], y), axis=0)
        unc_inv_cost_s = C_tilde.T @ cp.pos(prod_s - d[s])
        unc_rev_s = U.T @ cp.minimum(prod_s, d[s])

        constraints.append(unc_inv_cost_s - unc_rev_s - theta <= 0)
    
    constraints.append(cp.sum(y, axis=1) <= A)
    
    fixed_costs = cp.sum(cp.multiply(C, y))
    obj = cp.Minimize(fixed_costs + theta)
    prob = cp.Problem(obj,constraints)
    
    # solve problem
    time_limit = kwargs.get('time_limit', 2*60*60) - (time.time() - setup_time_start)
    if time_limit < 0:
        print("Error: did not provide sufficient time for setting up & solving problem")
        return (None, None)
    
#     prob.solve(solver=cp.MOSEK, mosek_params = {mosek.dparam.optimizer_max_time: time_limit})
    prob.solve(solver=cp.GUROBI, verbose=False, TimeLimit=time_limit)
    x_value = [theta.value, y.value] # Combine y and theta into 1 single solution vector
    return (x_value, prob.value)

def unc_obj_func(x, data, **kwargs):
    # extract values
    C = kwargs['C']
    C_tilde = kwargs['C_tilde']
    U = kwargs['U']
    d,p = data.T
    m,n = p[0].shape
    y = x[1]
    
    # compute obj function value:
    fixed_cost = np.sum(np.multiply(C, y))
    prod = [np.einsum('jk,jk->k', p[s], y) for s in range(len(data))]
    inventory_cost = np.array([np.dot(C_tilde, np.maximum(prod[s] - d[s],0)) for s in range(len(data))]) 
    revenue = np.array([np.dot(U, np.minimum(prod[s], d[s])) for s in range(len(data))]) 
    
    return fixed_cost + inventory_cost - revenue

def eval_x_OoS(x, obj, data, eval_unc_obj, **kwargs):
    unc_obj_func = eval_unc_obj['function']
    desired_rhs = eval_unc_obj['info']['desired_rhs']
    
    evals = unc_obj_func(x, data, **kwargs)  
    p_vio = sum(evals>(obj+(1e-6))) / len(data) 
    VaR = - np.quantile(evals, desired_rhs, method='inverted_cdf')
    return p_vio, VaR

In [ ]:
random_seed = 0
TIME_LIMIT = 1*60*60

In [ ]:
# provide functions and other info for generating & evaluating solutions
solve_SCP = solve_P_SCP
scale_dim_problem = 1
conf_param_alpha = 1e-9
risk_param_epsilon = 0.01

problem_instance = get_fixed_param_data(random_seed, scale_dim_problem=scale_dim_problem)
problem_instance['time_limit'] = TIME_LIMIT 

eval_unc_obj = {'function': unc_obj_func,
                'info': {'risk_measure': 'probability', # must be either 'probability' or 'expectation'
                         'desired_rhs': 1 - risk_param_epsilon}}

eval_unc_constr = None

In [ ]:
# Generate extra out-of-sample (OoS) data
random_seed = 1234
N_OoS = int(1e5)
data_OoS = generate_unc_param_data(1234, N_OoS, scale_dim_problem=scale_dim_problem)

In [ ]:
# classic approach:
random_seed = 0
dim_x = 5*scale_dim_problem * 10*scale_dim_problem
N_classic = util.determine_campi_N_min(dim_x, eval_unc_obj['info']['desired_rhs'], conf_param_alpha)
# N_classic = 100 #10580
data = generate_unc_param_data(random_seed, N_classic, scale_dim_problem=scale_dim_problem)

start_time = time.time()
x, obj = solve_P_SCP(data, **problem_instance)
runtime_classic = time.time() - start_time
x_classic = x
obj_classic = - obj
p_vio_classic, VaR_classic = eval_x_OoS(x, obj, data_OoS, eval_unc_obj, **problem_instance)

print(N_classic, runtime_classic, obj_classic, p_vio_classic, VaR_classic)

In [ ]:
# Care (2014) approach:
generate_unc_param_data = generate_unc_param_data
conf_param_alpha = 1e-9
dim_x = 5*scale_dim_problem * 10*scale_dim_problem
N_1 = 20 * dim_x
random_seed = 0

x, obj, N_2, runtime = util.solve_with_care2014(solve_SCP, problem_instance, generate_unc_param_data, 
                                                    eval_unc_obj, conf_param_alpha, dim_x, N_1=N_1,
                                                    random_seed=random_seed, scale_dim_problem=scale_dim_problem)
x_care = x
obj_care = - obj
p_vio_care, VaR_care = eval_x_OoS(x, obj, data_OoS, eval_unc_obj, **problem_instance)

print(N_1, N_2, runtime, obj_care, p_vio_care, VaR_care)

In [ ]:
# generate and split data into train and test
random_seed = 0
N_total = 10000
data = generate_unc_param_data(random_seed, N_total, scale_dim_problem=scale_dim_problem)

N_train = math.floor(N_total / 2)
data_train, data_test = train_test_split(data, train_size=N_train/N_total, random_state=random_seed)

In [ ]:
# run the algorithm
alg = iter_gen_and_eval_alg(solve_SCP, problem_instance, eval_unc_obj, eval_unc_constr, 
                            data_train, data_test, conf_param_alpha=conf_param_alpha,
                            verbose=False)
time_limit_alg = 60
stop_criteria={'max_elapsed_time': time_limit_alg} # in seconds (time provided to search algorithm)

N_test = math.ceil(N_total / 2)
desired_rhs = 1 - risk_param_epsilon
N2_min = alg._determine_N_min(N_test, desired_rhs)
eval_unc_obj['info']['N2_min'] = N2_min

(best_sol, runtime, num_iter, pareto_frontier, S_history) = alg.run(stop_criteria=stop_criteria)

obj_alg = - best_sol['obj']
p_vio_alg, VaR_alg = eval_x_OoS(best_sol['sol'], best_sol['obj'], data_OoS, eval_unc_obj, **problem_instance)
print(N_train, N_total-N_train, runtime, obj_alg, p_vio_alg, VaR_alg)

In [ ]:
num_iter

In [ ]:
S_history

# Now for the computational experiments

In [17]:
output_file_name = 'wdp_care2014_scale=2_eps=0.01_seeds=1-10'

headers = ['$dim(\mathbf{x})$', 'seed', 
           '$N$', '$T$', '$Obj.$', '$p_{vio}^{OoS}$', '$VaR^{OoS}$',
           '$N_1$', '$N_2$', '$T$', '$Obj.$', '$p_{vio}^{OoS}$', '$VaR^{OoS}$',
           '$N_1$', '$N_2$', '$T$', '$Obj.$', '$p_{vio}^{OoS}$', '$VaR^{OoS}$',
           '\#Iter.~(\\texttt{add})', '\#Iter.~(\\texttt{remove})', 
           '$\mu_{|\mathcal{S}_i|}$', '$\max_{i}|\mathcal{S}_i|$']

# Write headers to .txt file
with open(r'output/WeightedDistributionProblem/headers_'+output_file_name+'.txt','w+') as f:
    f.write(str(headers))

output_data = {}

random_seed_settings = [i for i in range(1, 11)]

# fixed info:
solve_SCP = solve_P_SCP

eval_unc_obj = {'function': unc_obj_func,
                    'info': {'risk_measure': 'probability'}}
eval_unc_constr = None
conf_param_alpha = 1e-9
scale_dim_problem = 2
dim_x = 5*scale_dim_problem * 10*scale_dim_problem

random_seed = 1234
N_OoS = int(1e5)
data_OoS = generate_unc_param_data(1234, N_OoS, scale_dim_problem=scale_dim_problem)

risk_param_epsilon = 0.01
eval_unc_obj['info']['desired_rhs'] = 1 - risk_param_epsilon

# N_classic = util.determine_campi_N_min(dim_x, 1-risk_param_epsilon, conf_param_alpha)

N_1 = 20 * dim_x # using the rule of thumb proposed in their paper
try:
    B_eps = sum(math.comb(N_1, i)*(risk_param_epsilon**i)*((1-risk_param_epsilon)**(N_1 - i)) for i in range(dim_x+1))
    N_2 = math.ceil((math.log(conf_param_alpha) - math.log(B_eps)) / math.log(1-risk_param_epsilon))
except OverflowError:
    # Equation (6) can be substituted by the handier formula:
    N_2 = math.ceil((1/risk_param_epsilon) * math.log(1/conf_param_alpha))

run_count = 0
for random_seed in random_seed_settings:

    problem_instance = get_fixed_param_data(random_seed, scale_dim_problem=scale_dim_problem)
    problem_instance['time_limit'] = 30*60 # maximum on SCP runtime (in seconds) 

#     # classic approach:
#     N_classic = 34918
#     data = generate_unc_param_data(random_seed, N_classic, scale_dim_problem=scale_dim_problem)
#     start_time = time.time()
#     x, obj = solve_P_SCP(data, **problem_instance)
#     runtime_classic = time.time() - start_time
#     obj_classic = - obj
#     p_vio_classic, VaR_classic = eval_x_OoS(x, obj, data_OoS, eval_unc_obj, **problem_instance)
    
    # FAST approach
    x, obj, N_2, runtime = util.solve_with_care2014(solve_SCP, problem_instance, generate_unc_param_data, 
                                                    eval_unc_obj, conf_param_alpha, dim_x, N_1=N_1, N_2=N_2,
                                                    random_seed=random_seed,
                                                    scale_dim_problem=scale_dim_problem)
    runtime_FAST = runtime 
    obj_FAST = - obj
    p_vio_FAST, VaR_FAST = eval_x_OoS(x, obj, data_OoS, eval_unc_obj, **problem_instance)
    
    # Our method
    N_total = min(N_1 + N_2, 10000)
    data = generate_unc_param_data(random_seed, N_total, scale_dim_problem=scale_dim_problem)
    N_train = math.floor(N_total / 2)
    data_train, data_test = train_test_split(data, train_size=(N_train/N_total), random_state=random_seed)
    
    alg = iter_gen_and_eval_alg(solve_SCP, problem_instance, eval_unc_obj, eval_unc_constr, 
                                data_train, data_test, conf_param_alpha=conf_param_alpha,
                                verbose=False)
    
#     stop_criteria={'max_elapsed_time': max(runtime_FAST, 5*60)} # in seconds (time provided to search algorithm)
    stop_criteria={'max_num_iterations': 200,
                    'max_elapsed_time': 30*60} 

    (best_sol, runtime, num_iter, pareto_frontier, S_history) = alg.run(stop_criteria=stop_criteria)
    
    obj_alg = - best_sol['obj']
    p_vio_alg, VaR_alg = eval_x_OoS(best_sol['sol'], best_sol['obj'], data_OoS, eval_unc_obj, **problem_instance)
    S_avg = sum(len(S_i) for S_i in S_history) / len(S_history)
    S_max = max(len(S_i) for S_i in S_history)
    
    output_data[(dim_x, random_seed)] = [#N_classic, runtime_classic, obj_classic, p_vio_classic, VaR_classic,
                                                    np.nan, np.nan, np.nan, np.nan, np.nan,
#                                         np.nan, np.nan, np.nan, np.nan, np.nan, np.nan,
#                                          np.nan, np.nan, np.nan, np.nan, np.nan, np.nan,
#                                          np.nan, np.nan, np.nan, np.nan]
                                                    N_1, N_2, runtime_FAST, obj_FAST, p_vio_FAST, VaR_FAST,
                                                    N_train, (N_total-N_train), runtime, obj_alg, p_vio_alg, VaR_alg,
                                                    num_iter['add'], num_iter['remove'], S_avg, S_max]

    output_file_name = 'new_output_data'
    with open(r'output/WeightedDistributionProblem/'+output_file_name+'.txt','w+') as f:
        f.write(str(output_data))

    run_count += 1
    print("Completed run: " + str(run_count))

Completed run: 1
Completed run: 2
Completed run: 3
Completed run: 4
Completed run: 5
Completed run: 6
Completed run: 7
Completed run: 8
Completed run: 9
Completed run: 10


In [18]:
output_file_name = 'wdp_care2014_scale=3_eps=0.01_seeds=1-10'

headers = ['$dim(\mathbf{x})$', 'seed', 
           '$N$', '$T$', '$Obj.$', '$p_{vio}^{OoS}$', '$VaR^{OoS}$',
           '$N_1$', '$N_2$', '$T$', '$Obj.$', '$p_{vio}^{OoS}$', '$VaR^{OoS}$',
           '$N_1$', '$N_2$', '$T$', '$Obj.$', '$p_{vio}^{OoS}$', '$VaR^{OoS}$',
           '\#Iter.~(\\texttt{add})', '\#Iter.~(\\texttt{remove})', 
           '$\mu_{|\mathcal{S}_i|}$', '$\max_{i}|\mathcal{S}_i|$']

# Write headers to .txt file
with open(r'output/WeightedDistributionProblem/headers_'+output_file_name+'.txt','w+') as f:
    f.write(str(headers))

output_data = {}

random_seed_settings = [i for i in range(1, 11)]

# fixed info:
solve_SCP = solve_P_SCP

eval_unc_obj = {'function': unc_obj_func,
                    'info': {'risk_measure': 'probability'}}
eval_unc_constr = None
conf_param_alpha = 1e-9
scale_dim_problem = 3
dim_x = 5*scale_dim_problem * 10*scale_dim_problem

random_seed = 1234
N_OoS = int(1e5)
data_OoS = generate_unc_param_data(1234, N_OoS, scale_dim_problem=scale_dim_problem)

risk_param_epsilon = 0.01
eval_unc_obj['info']['desired_rhs'] = 1 - risk_param_epsilon

# N_classic = util.determine_campi_N_min(dim_x, 1-risk_param_epsilon, conf_param_alpha)

N_1 = 20 * dim_x # using the rule of thumb proposed in their paper
try:
    B_eps = sum(math.comb(N_1, i)*(risk_param_epsilon**i)*((1-risk_param_epsilon)**(N_1 - i)) for i in range(dim_x+1))
    N_2 = math.ceil((math.log(conf_param_alpha) - math.log(B_eps)) / math.log(1-risk_param_epsilon))
except OverflowError:
    # Equation (6) can be substituted by the handier formula:
    N_2 = math.ceil((1/risk_param_epsilon) * math.log(1/conf_param_alpha))

run_count = 0
for random_seed in random_seed_settings:

    problem_instance = get_fixed_param_data(random_seed, scale_dim_problem=scale_dim_problem)
    problem_instance['time_limit'] = 30*60 # maximum on SCP runtime (in seconds) 

#     # classic approach:
#     N_classic = 34918
#     data = generate_unc_param_data(random_seed, N_classic, scale_dim_problem=scale_dim_problem)
#     start_time = time.time()
#     x, obj = solve_P_SCP(data, **problem_instance)
#     runtime_classic = time.time() - start_time
#     obj_classic = - obj
#     p_vio_classic, VaR_classic = eval_x_OoS(x, obj, data_OoS, eval_unc_obj, **problem_instance)
    
    # FAST approach
    x, obj, N_2, runtime = util.solve_with_care2014(solve_SCP, problem_instance, generate_unc_param_data, 
                                                    eval_unc_obj, conf_param_alpha, dim_x, N_1=N_1, N_2=N_2,
                                                    random_seed=random_seed,
                                                    scale_dim_problem=scale_dim_problem)
    runtime_FAST = runtime 
    obj_FAST = - obj
    p_vio_FAST, VaR_FAST = eval_x_OoS(x, obj, data_OoS, eval_unc_obj, **problem_instance)
    
    # Our method
    N_total = min(N_1 + N_2, 10000)
    data = generate_unc_param_data(random_seed, N_total, scale_dim_problem=scale_dim_problem)
    N_train = math.floor(N_total / 2)
    data_train, data_test = train_test_split(data, train_size=(N_train/N_total), random_state=random_seed)
    
    alg = iter_gen_and_eval_alg(solve_SCP, problem_instance, eval_unc_obj, eval_unc_constr, 
                                data_train, data_test, conf_param_alpha=conf_param_alpha,
                                verbose=False)
    
#     stop_criteria={'max_elapsed_time': max(runtime_FAST, 5*60)} # in seconds (time provided to search algorithm)
    stop_criteria={'max_num_iterations': 200,
                    'max_elapsed_time': 30*60} 

    (best_sol, runtime, num_iter, pareto_frontier, S_history) = alg.run(stop_criteria=stop_criteria)
    
    obj_alg = - best_sol['obj']
    p_vio_alg, VaR_alg = eval_x_OoS(best_sol['sol'], best_sol['obj'], data_OoS, eval_unc_obj, **problem_instance)
    S_avg = sum(len(S_i) for S_i in S_history) / len(S_history)
    S_max = max(len(S_i) for S_i in S_history)
    
    output_data[(dim_x, random_seed)] = [#N_classic, runtime_classic, obj_classic, p_vio_classic, VaR_classic,
                                                    np.nan, np.nan, np.nan, np.nan, np.nan,
#                                         np.nan, np.nan, np.nan, np.nan, np.nan, np.nan,
#                                          np.nan, np.nan, np.nan, np.nan, np.nan, np.nan,
#                                          np.nan, np.nan, np.nan, np.nan]
                                                    N_1, N_2, runtime_FAST, obj_FAST, p_vio_FAST, VaR_FAST,
                                                    N_train, (N_total-N_train), runtime, obj_alg, p_vio_alg, VaR_alg,
                                                    num_iter['add'], num_iter['remove'], S_avg, S_max]

    output_file_name = 'new_output_data_2'
    with open(r'output/WeightedDistributionProblem/'+output_file_name+'.txt','w+') as f:
        f.write(str(output_data))

    run_count += 1
    print("Completed run: " + str(run_count))

Completed run: 1
Completed run: 2
Completed run: 3
Completed run: 4
Completed run: 5
Completed run: 6
Completed run: 7
Completed run: 8
Completed run: 9
Completed run: 10


In [21]:
from numpy import nan

output_file_name = 'wdp_care2014_scale=2_eps=0.01_seeds=1-10_fast_robist'
# Read from .txt file
file_path = 'output/WeightedDistributionProblem/'+output_file_name+'.txt'
dic = ''
with open(file_path,'r') as f:
     for i in f.readlines():
        if i != "nan":
            dic=i #string
output_data_read = eval(dic)
output_data_read

{(200, 1): [nan,
  nan,
  nan,
  nan,
  nan,
  4000,
  2073,
  307.2479319572449,
  945.4921424634337,
  5e-05,
  998.8707391805626,
  3036,
  3037,
  146.04160284996033,
  987.103725570171,
  0.0045,
  996.9301714124102,
  127,
  73,
  59.95,
  86],
 (200, 2): [nan,
  nan,
  nan,
  nan,
  nan,
  4000,
  2073,
  323.34968996047974,
  960.818109879705,
  0.00012,
  1007.2422381194967,
  3036,
  3037,
  172.95060062408447,
  1004.9677675572764,
  0.00552,
  1011.9586117628464,
  134,
  66,
  67.79,
  110],
 (200, 3): [nan,
  nan,
  nan,
  nan,
  nan,
  4000,
  2073,
  329.2556335926056,
  938.7313969469382,
  0.00065,
  968.2823748545314,
  3036,
  3037,
  248.53921842575073,
  961.829979957286,
  0.00603,
  967.7871810665158,
  129,
  71,
  58.54,
  91],
 (200, 4): [nan,
  nan,
  nan,
  nan,
  nan,
  4000,
  2073,
  307.33293747901917,
  890.669601430684,
  6e-05,
  941.3629293759756,
  3036,
  3037,
  195.19799780845642,
  937.0491203840825,
  0.00394,
  947.6629492777394,
  133,
  67,

In [22]:
output_data = output_data_read
output_data

{(200, 1): [nan,
  nan,
  nan,
  nan,
  nan,
  4000,
  2073,
  307.2479319572449,
  945.4921424634337,
  5e-05,
  998.8707391805626,
  3036,
  3037,
  146.04160284996033,
  987.103725570171,
  0.0045,
  996.9301714124102,
  127,
  73,
  59.95,
  86],
 (200, 2): [nan,
  nan,
  nan,
  nan,
  nan,
  4000,
  2073,
  323.34968996047974,
  960.818109879705,
  0.00012,
  1007.2422381194967,
  3036,
  3037,
  172.95060062408447,
  1004.9677675572764,
  0.00552,
  1011.9586117628464,
  134,
  66,
  67.79,
  110],
 (200, 3): [nan,
  nan,
  nan,
  nan,
  nan,
  4000,
  2073,
  329.2556335926056,
  938.7313969469382,
  0.00065,
  968.2823748545314,
  3036,
  3037,
  248.53921842575073,
  961.829979957286,
  0.00603,
  967.7871810665158,
  129,
  71,
  58.54,
  91],
 (200, 4): [nan,
  nan,
  nan,
  nan,
  nan,
  4000,
  2073,
  307.33293747901917,
  890.669601430684,
  6e-05,
  941.3629293759756,
  3036,
  3037,
  195.19799780845642,
  937.0491203840825,
  0.00394,
  947.6629492777394,
  133,
  67,

In [23]:
# obtain average and std dev
import pandas as pd
df_output = pd.DataFrame.from_dict(output_data, orient='index')
df_output

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
"(200, 1)",NaN,NaN,NaN,NaN,NaN,4000,2073,307.247932,945.492142,0.00005,...,3036,3037,146.041603,987.103726,0.00450,996.930171,127,73,59.95,86
"(200, 2)",NaN,NaN,NaN,NaN,NaN,4000,2073,323.349690,960.818110,0.00012,...,3036,3037,172.950601,1004.967768,0.00552,1011.958612,134,66,67.79,110
"(200, 3)",NaN,NaN,NaN,NaN,NaN,4000,2073,329.255634,938.731397,0.00065,...,3036,3037,248.539218,961.829980,0.00603,967.787181,129,71,58.54,91
"(200, 4)",NaN,NaN,NaN,NaN,NaN,4000,2073,307.332937,890.669601,0.00006,...,3036,3037,195.197998,937.049120,0.00394,947.662949,133,67,67.55,106
"(200, 5)",NaN,NaN,NaN,NaN,NaN,4000,2073,344.433322,961.356275,0.00171,...,3036,3037,149.396589,977.751421,0.00615,984.089265,125,75,51.42,75
"(200, 6)",NaN,NaN,NaN,NaN,NaN,4000,2073,306.051160,985.608148,0.00061,...,3036,3037,191.001730,1010.680159,0.00398,1021.526356,127,73,62.76,104
"(200, 7)",NaN,NaN,NaN,NaN,NaN,4000,2073,324.039705,924.900160,0.00003,...,3036,3037,156.323200,982.977785,0.00680,987.710911,124,76,55.47,92
"(200, 8)",NaN,NaN,NaN,NaN,NaN,4000,2073,326.188021,965.932856,0.00094,...,3036,3037,187.779819,982.403128,0.00512,990.629837,128,72,62.79,96
"(200, 9)",NaN,NaN,NaN,NaN,NaN,4000,2073,343.863430,947.325524,0.00043,...,3036,3037,172.910794,966.995788,0.00252,982.560364,128,72,64.27,101
"(200, 10)",NaN,NaN,NaN,NaN,NaN,4000,2073,345.106352,971.522487,0.00032,...,3036,3037,169.526377,999.711685,0.00415,1010.269194,126,74,61.51,98


In [24]:
df_output.mean()

0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
5     4000.000000
6     2073.000000
7      325.686818
8      949.235670
9        0.000492
10     988.731695
11    3036.000000
12    3037.000000
13     178.966793
14     981.147056
15       0.004871
16     990.112484
17     128.100000
18      71.900000
19      61.205000
20      95.900000
dtype: float64

In [ ]:
output_data_str = {}
for i,res in output_data.items():
    res_str = []
    for i2,el in enumerate(res):
        i3 = i2-5
        if np.isnan(el):
            res_str.append('-')
                
        elif i3 in [0,1,6,7]:
            res_str.append(f'{round(df_output.iloc[:,i2].mean(),0):.0f}') 
        elif i3 in [2,8]:
            res_str.append(f'{round(df_output.iloc[:,i2].mean(),0):.0f}' + " ("+f'{round(df_output.iloc[:,i2].std(),0):.0f}'+")")
        elif i3 in [12,13,14,15]:
            res_str.append(f'{round(df_output.iloc[:,i2].mean(),1):.1f}' + " ("+f'{round(df_output.iloc[:,i2].std(),1):.1f}'+")")
        elif i3 in [3,5,9,11]:
            res_str.append(f'{round(df_output.iloc[:,i2].mean(),2):.2f}' + " ("+f'{round(df_output.iloc[:,i2].std(),2):.2f}'+")")
        else:
            res_str.append(f'{round(df_output.iloc[:,i2].mean(),4):.4f}' + " ("+f'{round(df_output.iloc[:,i2].std(),4):.4f}'+")")
    
    output_data_str[i] = res_str
    break

In [ ]:
output_data_str

In [ ]:
headers = ['dim_x', 'seed', 
           '$N^{Classic}$', '$T^{Classic}$', '$Obj.~(Classic)$', '$p_{vio}^{OoS}~(Classic)$', '$VaR^{OoS}~(Classic)$',
           '$N_1$', '$N_2$', '$T$', '$Obj.$', '$p_{vio}^{OoS}$', '$VaR^{OoS}$',
           '$N_1$', '$N_2$', '$T$', '$Obj.$', '$p_{vio}^{OoS}$', '$VaR^{OoS}$',
           '\#Iter.~(\\texttt{add})', '\#Iter.~(\\texttt{remove})', 
           '$\mu_{|\mathcal{S}_i|}$', '$\max_{i}|\mathcal{S}_i|$']

In [ ]:
import dataio
dataio.write_output_to_latex(2, headers, output_data_str)